# 情報科学演習II アルゴリズムとデータ構造II 第2週 4冊目

#### 第2週目次

1. ダイクストラのアルゴリズムの簡単な実装（1冊目）
2. 実装のクラス化（2冊目）
3. 優先度付きキュー（3冊目）
4. ダイクストラのアルゴリズムの効率的な実装（4冊目）

今週4冊目のノートブックです．
同一の仮想マシンで混乱するのを防ぐ目的で別のノートブックにしています．
2冊目と3冊目のノートブックを見やすい場所に表示しておきましょう．

## 4. ダイクストラのアルゴリズムの効率的な実装

この章では，ダイクストラのアルゴリズムを効率的に実装したSecondImplementationクラスを作成します．
2章で実装したダイクストラのアルゴリズムに優先度付きキューを組み込んで高速化します．
優先度付きキューの使い方は3章で学びました．

次の2つのセルは準備ですので，実行してください．
二つ目のセルで必要なモジュールを`import`し，いくつかの関数を定義しておきます．

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz
# 最終行は Successfully installed pygraphviz-x.xx
# x.xxの部分は1.13などのバージョン番号

In [ ]:
import networkx as nx
import math
import pygraphviz as pgv
from IPython.display import SVG, display_svg
from test import support # sortdict関数

# グラフの頂点のリストと辺のリストを出力する関数
def print_graph(G): # GはnetworkxのGraphクラス (networkx.classes.graph.Graphクラス)
  print('print_graph:')
  print('  nodes', list(G.nodes))
  print('  edges', list(G.edges)) # 辺の重みも出力するにはedgesの代わりにedges.data()
  
# グラフを図示する関数
def draw_graph(G):
  A = nx.nx_agraph.to_agraph(G)
  for edge in G.edges(data='weight'):
    u, v, d = edge
    A.get_edge(u, v).attr['label'] = d

  A.layout(args='-Nshape=circle')  # default prog='neato' 
  A.draw('tmp.svg')
  display_svg(SVG('tmp.svg'))

# グラフのサンプル
def sample_graph1():
  G = nx.Graph() # 無向グラフ
  for i,j,w in [[0, 1, 5], [0, 4, 9], [0, 7, 8], [1, 2, 12], [1, 3, 15], 
                [1, 7, 4], [2, 3, 3], [2, 6, 11], [3, 6, 9], [4, 5, 4], 
                [4, 7, 5], [5, 2, 1], [5, 6, 13], [7, 2, 7], [7, 5, 6]]:
    G.add_edge(i, j, weight=w) # weightはnetworkx, labelはgraphviz
  return G

G1 = sample_graph1()
L1 = nx.get_edge_attributes(G1, 'weight') # w1は辞書型でキーは辺，値は重み（辺の長さ），
answer_d1 = nx.single_source_dijkstra_path_length(G1, 0)
print('G1の辺の長さ L1 = ', L1)
print('グラフ G1')
draw_graph(G1)

def sample_graph2():
  G = nx.Graph() # 無向グラフ
  for i,j,w in [[7, 5, 6], [7, 2, 7], [5, 6, 13], [5, 2, 1], [4, 7, 5], 
                [4, 5, 4], [3, 6, 9], [2, 6, 11], [2, 3, 3], [1, 7, 4], 
                [1, 3, 15], [1, 2, 12], [0, 7, 8], [0, 4, 9], [0, 1, 5]]:
    G.add_edge(i, j, weight=w) # weightはnetworkx, labelはgraphviz
  L = nx.get_edge_attributes(G, 'weight')
  print('edge length', w)
  return G

G2 = sample_graph2()
L2 = nx.get_edge_attributes(G2, 'weight')
answer_d2 = {0: 0, 1: 5, 2: 14, 3: 17, 4: 9, 5: 13, 6: 25, 7: 8}
print('G2の辺の長さ L2 = ', L2)
print('グラフ G2')
draw_graph(G2)

### 4.1 実装

次のような方針で効率的なダイクストラのアルゴリズムを実装します．

1. クラス名を`SecondImplementation`にします．
1. 2章で作成した`FirstImplemetaion`クラスと3章で作成した`DijkstraHeap`クラスの関数をコピーし，必要な変更を加えます．
1. `__init__`で初期化するデータ構造は，`FirstImplementation`クラスで使った`d`，`Q`，`pi`に加えて，`DijkstraHeap`クラスの`pos`の4つです．
1. 次の2つの関数は`FirstImplemetaion`クラスの同名の関数を使います．
   - `dijkstra`関数
   - `initialize_single_source`関数
1. 次の2つの関数は`FirstImplemetaion`クラスと`DijkstraHeap`クラスの両方にあります．
   優先度キューを実装しているので`DijkstraHeap`クラスのものを使います．
   - `extract_min`関数
   - `insert`関数
1. `relax`関数は`FirstImplemetaion`クラスの同名の関数を使います．
   ただし，距離が短くなった頂点についてヒープ条件を満たすとは限らないので，ヒープ条件を維持するために`d[v] > d[u] + luv`が成り立つならば`self.heapify_up(self.pos[v])`を呼び出します．
   `if`文の部分が次のようになります．

   ```
       if d[v] > d[u] + luv:
         d[v] = d[u] + luv
         pi[v] = u
         self.heapify_up(self.pos[v]) # この行を追加
   ```

1. 次の3つの関数は`DijkstraHeap`クラスの同名の関数を使います．
   - `heapify_up`関数
   - `heapify_down`関数
   - `draw`関数

1. `dijkstra`関数の`d[s] = 0`の行で頂点sの距離が短くなりますので，
   頂点`s`がヒープ条件を満たさない位置にあるかもしれません．
   この行の後に，節点sをヒープ条件を満たす位置に移動する処理が必要です．
   `heapify_up`関数でこの処理が行えます．

動作がおかしい場合は`dijkstra`関数の適当なところに`self.draw()`関数を追加して，ヒープの状態を確認するといいでしょう．

**練習**
次のコードセルの`....`の部分を上の方針に沿って適切に書き，`SecondImplementation`クラスを定義しなさい．

In [ ]:
class SecondImplementation:
    
  def __init__(self):
    self.d = {}   # d[u]は頂点uの距離
    self.Q = []   # 頂点が距離順に格納されたヒープ．Q[0]が最小距離の頂点
    self.pos = {} # pos[u]は頂点uのヒープQの位置(添字)．Q[pos[u]]に頂点uが格納されている．
    self.pi = {}  # pi[u]は頂点uへの最短経路上で，頂点uの直前にある頂点

  def dijkstra(self, G, L, s):
    ....
    return self.d

  def initialize_single_source(self, G):
    ....

  def insert(self, u): # ヒープに頂点uを追加する
    ....

  def extract_min(self): # ヒープから最短距離の頂点を取り出す
    ....

  def relax(self, u, v, L):
    ....

  def heapify_up(self, i): # ヒープ条件を満たすように，ヒープのi番目の頂点を上に移動する
    ....

  def heapify_down(self, i): # ヒープ条件を満たすように，ヒープのi番目の頂点を下に移動する
    ....

  def draw(self, filename = 'tmp'): # ヒープを図示する．ヒープの動作を確認するための関数
    ....


### 4.2 テスト

うまく`SecondImplementation`クラスが作れた確かめましょう．

**練習**
次のコードセルを実行して，
`FirstImplementation`クラスと同じ結果が得られることを確かめよ．

`d = {0: 0, 1: 5, 2: 14, 3: 17, 4: 9, 5: 13, 6: 25, 7: 8}`が正解です．

In [ ]:
log = False # log = Trueにすると途中経過が出力される

c = SecondImplementation()
s = 0
d = c.dijkstra(G1, L1, s)
print("d =", support.sortdict(d)) # 距離の辞書dを出力（頂点でソートして出力)
if d == answer_d1:
  print('すべての頂点の距離が正しく求められました．')
else:
  print('距離が間違っている場合があります．')
  print()
  for u, du in d.items():
    if du != answer_d1[u]:
      print(F'始点{s}から頂点{u}への最短距離は{answer_d1[u]}ですが，{du}と求めています．')
  draw_graph(G1)

**練習**
もう一つのグラフについても以下のコードセルを実行して確かめましょう．

`d = {0: 0, 1: 7, 2: 12, 3: 15, 4: 9, 5: 11, 6: 23, 7: 5}`が正解です．

In [ ]:
log = False # log = Trueにすると途中経過が出力される

c = SecondImplementation()
s = 0
d = c.dijkstra(G2, L2, s)
print("d =", support.sortdict(d)) # 距離の辞書dを出力（頂点でソートして出力)
if d == answer_d2:
  print('すべての頂点の距離が正しく求められました．')
else:
  print('距離が間違っている場合があります．')
  print()
  for u, du in d.items():
    if du != answer_d2[u]:
      print(F'始点{s}から頂点{u}への最短距離は{answer_d1[u]}ですが，{du}と求めています．')
  draw_graph(G2)

**練習**
最後にグラフG1について，始点を変えながら動作を確かめましょう．

In [ ]:
for s in range(G1.order()):
  d = SecondImplementation().dijkstra(G1, L1, s)
  answer_d = nx.single_source_dijkstra_path_length(G1, s)
  if d == answer_d:
    print(F'始点が{s}のときすべての頂点の距離が正しく求められました．')
  else:
    print(F'始点が{s}のとき距離が間違っている頂点があります．')
    print()
    for u, du in d.items():
      if du != answer_d[u]:
        print(F'始点{s}から頂点{u}への最短距離は{answer_d[u]}ですが，{du}と求めています．')

draw_graph(G1)

以上です．次回は二つの実装の実行時間を計ります．